<a href="https://colab.research.google.com/github/leenajoshi1/bbit-learning-labs/blob/main/TCG_x_TicketCity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import os

os.environ['USE_AUTH_EPHEM'] = '0'  #Bypass the special permissions error from before

# Authenticate to Google Cloud
auth.authenticate_user()
project_id = 'ticketcity-tcg' # Replace with your project ID
client = bigquery.Client(project=project_id) # This line is added to create the client object
print('Authenticated')

!pip install google-cloud-bigquery

project_id = 'ticketcity-tcg'

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

# Replace with your project ID and dataset/table names.
# dataset_id = ''
table_id = 'data-ticketcity.TC_Data.InventoryStream'

# Create a SQL query string.
query = f"""
    WITH PriceChanges AS (
    SELECT
        EventID,
        BlockId,
        Price,
        Time,  -- This is in Unix time (seconds)
        LEAD(Time) OVER (PARTITION BY EventID, BlockId ORDER BY Time) AS next_Time,
        LEAD(Price) OVER (PARTITION BY EventID, BlockId ORDER BY Time) AS next_price,
        TIMESTAMP_DIFF(
            TIMESTAMP_SECONDS(LEAD(Time) OVER (PARTITION BY EventID, BlockId ORDER BY Time)),
            TIMESTAMP_SECONDS(Time),
            MINUTE
        ) AS time_difference
    FROM
        `data-ticketcity.TC_Data.InventoryStream`
),
Stats AS (
    -- Calculate mean and standard deviation for time differences
    SELECT
        AVG(time_difference) AS avg_time_difference,
        STDDEV(time_difference) AS stddev_time_difference
    FROM
        PriceChanges
    WHERE
        time_difference IS NOT NULL
),
ConfidenceInterval AS (
    -- Define confidence interval bounds (95% confidence interval)
    SELECT
        avg_time_difference,
        stddev_time_difference,
        avg_time_difference - 1.96 * stddev_time_difference AS lower_bound, -- edit critical value of z score as needed, currently at 95%
        avg_time_difference + 1.96 * stddev_time_difference AS upper_bound
    FROM
        Stats
)
-- Join and display results with confidence interval and confidence percentage
SELECT
    pc.EventID,
    pc.BlockId,
    pc.Time,
    pc.Price,
    pc.next_Time,
    pc.next_price,
    pc.time_difference,
    (pc.next_price - pc.Price) AS price_difference,
    ci.lower_bound,
    ci.upper_bound,
    -- Calculate confidence interval range
    ci.upper_bound - ci.lower_bound AS confidence_interval_range,
    -- Calculate confidence percentage
    CASE
        WHEN ABS(pc.time_difference - ci.avg_time_difference) <= (ci.upper_bound - ci.lower_bound)
        THEN ROUND((1 - (ABS(pc.time_difference - ci.avg_time_difference) / (ci.upper_bound - ci.lower_bound))) * 100, 2)
        ELSE 0
    END AS confidence_percentage,
    CASE
        WHEN pc.time_difference >= ci.lower_bound
             AND pc.time_difference <= ci.upper_bound
        THEN 'Automatic'
        ELSE 'Manual'
    END AS update_type
FROM
    PriceChanges pc, ConfidenceInterval ci
WHERE
    pc.time_difference <= 30  -- Within 30-minute interval
    AND ABS(pc.next_price - pc.Price) > 0  -- Price changes occurred
    AND pc.time_difference IS NOT NULL
LIMIT 50;
"""

# Execute the query.
query_job = client.query(query)

# Fetch the results.
results = query_job.result()

# Print the results.
for row in results:
  print(row.BlockId)


Authenticated
VB10160820635
VB10160820635
VB10160820635
VB9977705755
VB9977705755
VB9977705755
VB9996473822
VB9996473822
VB9996473822
VB9996473822
VB9996473822
VB9996473822
VB10246627661
VB10012591858
VB10188228084
VB9816100778
VB9995424121
VB10005833258
VB10023316189
VB10015483647
VB9976767732
VB10039229214
VB10088895900
VB10702477032
VB10157225852
VB10151085445
VB10176738828
VB10013797781
VB10013797781
VB10077645292
VB10077645292
VB10199109449
VB10199109449
VB10199109449
VB10199109449
VB10199109449
VB10199109449
VB10202531363
VB10202531363
VB10202531363
VB10202531363
VB10398933406
VB10398709397
VB10456105437
VB10006809217
VB9818497893
VB10126959252
VB10665475108
VB10563765106
VB10205241365
